# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv("pokemon.csv")

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [6]:
# your code here
pokemon.head()
#from the basic inspection of the data, it is likely that total is derived as a linear combination from...
#...the values of the columns "HP", "Attack", "Defense", "Sp.Atk", "Sp.Def", "Speed".

#to test this I will construct a regression model, splitting the data in train and test in order to test it

#selecting the df I need:
pokemon_reg = pokemon[["Total","HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]]
pokemon_reg


,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,318,45,49,49,65,65,45
1,405,60,62,63,80,80,60
2,525,80,82,83,100,100,80
3,625,80,100,123,122,120,80
4,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...
795,600,50,100,150,100,150,50
796,700,50,160,110,160,110,110
797,600,80,110,60,150,130,70
798,680,80,160,60,170,130,80


In [9]:
#check if NaNs:
display(pokemon_reg.isna().sum())

Total      0
HP         0
Attack     0
Defense    0
Sp. Atk    0
Sp. Def    0
Speed      0
dtype: int64

In [7]:
# Build X and y
y = pokemon_reg['Total']
X = pokemon_reg.drop(['Total'], axis=1)

In [10]:
# split train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# scale numericals
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import StandardScaler

# Normalizing data
transformer = MinMaxScaler().fit(X_train) # will reuse this transformer for X_test later
X_train_scaled_arr = transformer.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled_arr, columns=X_train.columns)
X_train_scaled.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.367347,0.378378,0.488889,0.434783,0.452381,0.457143
1,0.387755,0.729730,0.222222,0.108696,0.166667,0.514286
2,0.163265,0.540541,0.533333,0.244565,0.357143,0.257143
3,0.224490,0.367568,0.222222,0.201087,0.261905,0.457143
4,0.346939,0.097297,0.191111,0.070652,0.133333,0.102857


In [15]:
#Linear regression model
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(X_train_scaled,y_train)

LinearRegression()

In [16]:
#To test I will first need to apply scaler to X_test:
X_test_scaled_arr = transformer.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled_arr, columns=X_test.columns)
X_test_scaled.head(10)

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.334694,0.540541,0.377778,0.625000,0.333333,0.531429
1,0.265306,0.378378,0.311111,0.625000,0.357143,0.200000
2,0.183673,0.351351,0.177778,0.326087,0.142857,0.314286
3,0.163265,0.324324,0.453333,0.516304,0.414286,0.462857
4,0.224490,0.324324,0.266667,0.217391,0.142857,0.485714
5,0.224490,0.459459,0.488889,0.190217,0.452381,0.302857
6,0.367347,0.367568,0.346667,0.342391,0.300000,0.285714
7,0.163265,0.470270,0.457778,0.445652,0.419048,0.171429
8,0.163265,0.297297,0.244444,0.271739,0.190476,0.142857
9,0.204082,0.254054,0.724444,0.201087,0.561905,0.142857


In [17]:
#Model Validation
#Importing libraries
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import math
#Computing validation metrics
predictions_test = lm.predict(X_test_scaled)
r2_score(y_test, predictions_test)

1.0

In [18]:
#MSE
mse=mean_squared_error(y_test,predictions_test)
print("MSE = ", mse)
#RMSE
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
print("RMSE = ", rmse)
#MAE
mae = mean_absolute_error(y_test, predictions_test)
print("MAE = ", mae)


MSE =  1.516127550962367e-26
RMSE =  1.2313113135849793e-13
MAE =  9.467981954003334e-14


### Observations:
- The high value of the R2 and the validation metrics strongly suggest the column 'total' is indeed a linear combination of the columns "HP", "Attack", "Defense", "Sp.Atk", "Sp.Def", "Speed".
- The formula provided by the model, however, does not apply to all pokemons. 
- In fact, on average, the formula provided by the model delivers an error of 1.2 points when estimating 'Total', as shown in the RMSE calculation in the tested data

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [25]:
# encoding type 1 and type 2
#creating a subset with these two columns
pokemon_type1 = pokemon[['Type 1']]
pokemon_type1
pokemon_type2 = pokemon[['Type 2']]
pokemon_type2

,Type 2
0,Poison
1,Poison
2,Poison
3,Poison
4,NaN
...,...
795,Fairy
796,Fairy
797,Ghost
798,Dark


In [24]:
# importing the library
from sklearn.preprocessing import OneHotEncoder
#encoding first the type_1 column:
encoder_1 = OneHotEncoder().fit(pokemon_type1)
encoded_pok1 = encoder_1.transform(pokemon_type1).toarray()
cols = encoder_1.get_feature_names_out(input_features=pokemon_type1.columns)
onehot_encoded1 = pd.DataFrame(encoded_pok1, columns=cols)
onehot_encoded1.head()
#deleting the 'Type_1' in column names:
converter = lambda x: x.replace('Type 1_', '')
col= onehot_encoded1.columns
col = list(map(converter, col))
onehot_encoded1.columns = col
onehot_encoded1.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
#encoding the type_2 column:
encoder_2 = OneHotEncoder().fit(pokemon_type2)
encoded_pok2 = encoder_2.transform(pokemon_type2).toarray()
cols = encoder_2.get_feature_names_out(input_features=pokemon_type2.columns)
onehot_encoded2 = pd.DataFrame(encoded_pok2, columns=cols)
#deleting the 'Type_2' in column names:
converter = lambda x: x.replace('Type 2_', '')
col= onehot_encoded2.columns
col = list(map(converter, col))
onehot_encoded2.columns = col
onehot_encoded2.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
#checking same lenght
display(onehot_encoded1.shape)
display(onehot_encoded2.shape)
#deleting nan from encoded2
onehot_encoded2 = onehot_encoded2.drop(['nan'], axis=1)
display(onehot_encoded2.shape)

(800, 18)

(800, 19)

(800, 18)

In [31]:
#Now combining the two dataframes
onehot_total = onehot_encoded1+onehot_encoded2
onehot_total.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [33]:
# First I will concatenate Total ('Y') with the onehot_total
Total_onehot = pd.concat([pokemon[['Total']], onehot_total], axis=1)
Total_onehot.head()
#Now will compute the correlation matrix and extract correlation with total
correlations_matrix = Total_onehot.corr()
correlation_total = correlations_matrix['Total'].sort_values()
correlation_total


Bug        -0.145781
Normal     -0.105331
Poison     -0.090441
Grass      -0.052592
Fairy      -0.036698
Water      -0.021665
Ghost       0.003641
Ground      0.015060
Electric    0.020971
Rock        0.032731
Dark        0.056154
Flying      0.059383
Ice         0.060248
Fighting    0.077786
Fire        0.078726
Steel       0.109703
Psychic     0.124688
Dragon      0.229705
Total       1.000000
Name: Total, dtype: float64

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here
#Pokemos with type Dragon, Psychic or Steel